# A Collection of Results from OpenWPM data

To use, enter in the `datadir_path` bellow, then run all cells

In [ ]:
from pathlib import Path
datadir_path : Path = Path('/home/ndanner_plp/OpenWPM/crawl-data/datadir-0-10000')

In [ ]:
import sys
import os
from sqlalchemy.engine import Engine
from sqlalchemy import create_engine, URL 
import plyvel
from typing import Any
%reload_ext sql

parent_path : Path = Path(sys.path[0]).parent.resolve()
if str(parent_path) not in sys.path:
    sys.path.append(str(parent_path))


database_url : URL = URL.create(drivername = "sqlite", database = str(datadir_path.joinpath("crawl-data.sqlite")) )
os.environ["DATABASE_URL"] = str(database_url)
engine : Engine = create_engine(database_url)
# if 'db' not in locals():
#     db : Any = plyvel.DB( str(datadir_path.joinpath("leveldb")) ) #type: ignore

### Prevalence of `CanvasRenderingContext2D` versus `WebGLRenderingContext`

In [ ]:
%%sql
WITH a AS (
    SELECT COUNT(DISTINCT visit_id) AS Total
    FROM http_responses WHERE 
    response_status LIKE "2%"
), b AS (
    SELECT COUNT(DISTINCT visit_id) AS Using_Canvas
    FROM javascript
    WHERE symbol LIKE "CanvasRenderingContext2D%"
), c AS (
    SELECT COUNT(DISTINCT visit_id) AS Using_WEBGL
    FROM javascript
    WHERE symbol LIKE "WebGLRenderingContext%"
)
SELECT a.Total, b.Using_Canvas, c.Using_WEBGL
FROM a, b, c

### Signs of Canvas fingerprinting among the group classified as Canvas font fingerprinting dynamically but not lexically

In [ ]:
%%sql 
WITH domain AS(
SELECT *
FROM javascript AS js
WHERE (js.visit_id, js.script_url) IN (SELECT visit_id, script_url FROM analysis_results WHERE Canvas_Font_1M_Dynamic = True AND Canvas_Font_1M_Static = False )
),
total AS(
SELECT COUNT(*) AS total
FROM ( SELECT DISTINCT visit_id, script_url
    FROM domain
)
),
b AS(
SELECT COUNT(*) AS using_fillText_questionably
FROM ( SELECT * FROM 
    domain AS js
    WHERE js.symbol = "CanvasRenderingContext2D.fillText" 
    AND ( js.arguments LIKE '["Cwm fjordbank glyphs vext quiz%' OR js.arguments LIKE '["Hel$&?6\%){mZ+#@%'  ESCAPE '\'  
    )
    GROUP BY js.visit_id, js.script_url
)
)
SELECT *
FROM total, b

### Use of the Canvas API among the ground classified as Canvas font fingerprinting lexically but not dynamically

In [ ]:
%%sql 
WITH domain AS(
SELECT visit_id, script_url 
FROM analysis_results 
WHERE Canvas_Font_1M_Dynamic = False AND Canvas_Font_1M_Static = True 
),
total AS(
SELECT COUNT(*) AS total
FROM domain
),
b AS(
SELECT COUNT(*) AS using_Canvas_API
FROM ( 
    SELECT visit_id, script_url 
    FROM javascript 
    WHERE (visit_id, script_url) IN domain
    AND symbol  LIKE "CanvasRenderingContext2D%" 
    GROUP BY visit_id, script_url
)
),
c AS(
SELECT COUNT(*) AS using_MeasureText_and_font
FROM ( 
    SELECT visit_id, script_url 
    FROM domain 
    WHERE (visit_id, script_url) IN (SELECT visit_id, script_url FROM javascript WHERE symbol = "CanvasRenderingContext2D.font")
    AND  (visit_id, script_url) IN (SELECT visit_id, script_url FROM javascript WHERE symbol = "CanvasRenderingContext2D.measureText")
)
),
d AS(
    SELECT COUNT(*) AS AD_Network_using_MeasureText_and_font
    FROM (
        SELECT visit_id, script_url
        FROM domain
        WHERE (visit_id, script_url) IN (SELECT visit_id, script_url FROM javascript WHERE symbol = "CanvasRenderingContext2D.font")
        AND  (visit_id, script_url) IN (SELECT visit_id, script_url FROM javascript WHERE symbol = "CanvasRenderingContext2D.measureText")
        AND script_url LIKE "https://cdn.taboola.com/libtrc/cta-component%"
    )
)
SELECT *
FROM total, b, c, d

### Signs of Canvas fingerprinting among the group classified as WebGL parameter fingerprinting dynamically but not lexically

In [ ]:
%%sql
WITH domain AS(
SELECT visit_id, script_url 
FROM analysis_results 
WHERE WebGL_Dynamic = True AND WebGL_Static = False 
),
total AS(
SELECT COUNT(*) AS total
FROM domain
),
b AS(
SELECT COUNT(*) AS using_fillText_questionably
FROM ( 
    SELECT visit_id, script_url 
    FROM javascript 
    WHERE (visit_id, script_url) IN domain
    AND symbol = "CanvasRenderingContext2D.fillText" 
    AND arguments LIKE '["Cwm fjordbank glyphs vext quiz%'
    GROUP BY visit_id, script_url
)
)
SELECT *
FROM total, b

### Signs of Canvas fingerprinting among the group classified as WebGL parameter fingerprinting lexically but not dynamically

In [ ]:
%%sql
WITH domain AS(
SELECT visit_id, script_url 
FROM analysis_results 
WHERE WebGL_Dynamic = False AND WebGL_Static = True 
),
total AS(
SELECT COUNT(*) AS total
FROM domain
),
b AS(
SELECT COUNT(*) AS using_Canvas_API
FROM ( 
    SELECT visit_id, script_url 
    FROM javascript 
    WHERE (visit_id, script_url) IN domain
    AND symbol  LIKE "CanvasRenderingContext2D%" 
    GROUP BY visit_id, script_url
)
),
c AS (
SELECT COUNT(*) AS using_WebGL_API
FROM (
    SELECT visit_id, script_url 
    FROM javascript 
    WHERE (visit_id, script_url) IN domain
    AND symbol  LIKE "WebGLRenderingContext%" 
    GROUP BY visit_id, script_url
)
),
d AS(
SELECT COUNT(*) AS using_fillText_questionably
FROM ( 
    SELECT visit_id, script_url 
    FROM javascript 
    WHERE (visit_id, script_url) IN domain
    AND symbol = "CanvasRenderingContext2D.fillText" 
    AND arguments LIKE '["Cwm fjordbank glyphs vext quiz%'
    GROUP BY visit_id, script_url
)
),
e AS (
SELECT COUNT(*) AS classified_As_Canvas_Fingerprinting_Dynamically
FROM(
    SELECT *
    FROM domain
    WHERE (visit_id, script_url) IN (SELECT visit_id, script_url FROM analysis_results WHERE Canvas_1M_Dynamic = True)
)
)
SELECT *
FROM total, b, c, d, e